In [ ]:
!pip -q install transformers
!pip -q install sentencepiece==0.1.94
!pip -q install datasets
!wget -q "https://storage.googleapis.com/totto/totto_data.zip"
!unzip -q "totto_data.zip"

     |████████████████████████████████| 2.1MB 7.4MB/s 
     |████████████████████████████████| 901kB 38.5MB/s 
     |████████████████████████████████| 3.3MB 56.9MB/s 
     |████████████████████████████████| 1.1MB 7.8MB/s 
     |████████████████████████████████| 225kB 9.2MB/s 
     |████████████████████████████████| 112kB 16.0MB/s 
     |████████████████████████████████| 245kB 15.4MB/s 


In [ ]:
!git clone https://github.com/google-research/language.git language_repo

Cloning into 'language_repo'...
remote: Enumerating objects: 2105, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 2105 (delta 84), reused 227 (delta 69), pack-reused 1856
Receiving objects: 100% (2105/2105), 2.38 MiB | 9.96 MiB/s, done.
Resolving deltas: 100% (1179/1179), done.


In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/Totto/Totto_with_tags.csv")
df=df[:100]
# df=df.sample(frac=1).reset_index()

In [ ]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import AdamW, WarmUp, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
MAXLENI=400
MAXLENO=200

In [ ]:
%cd language_repo
!python3 -m language.totto.create_table_to_text_html --input_path="/content/totto_data/totto_train_data.jsonl" --output_dir="."

/content/language_repo
Processed 100 examples
Num examples processed: 100


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model=T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

In [ ]:
special_tokens_dict = {'pad_token': '<pad>', 'bos_token': '<bos>', 'eos_token': '<eos>', 
                       'additional_special_tokens': ['<PAGESTART>', '<PAGEEND>', '<SECTIONSTART>', '<SECTIONEND>',
                                                     '<TABLESTART>','<TABLEEND>','<CELLSTART>','<CELLEND>','<COLHEADERSTART>',
                                                     '<COLHEADEREND>','<ROWHEADERSTART>','<ROWHEADEREND>']}

num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

print('We have added', num_added_toks, 'tokens')
model.encoder.resize_token_embeddings(len(tokenizer))
model.decoder.resize_token_embeddings(len(tokenizer))

We have added 14 tokens


Embedding(32114, 768)

In [ ]:
# model=T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)
model.load_state_dict(torch.load("/content/drive/MyDrive/Totto/totto_model_6_base_new_tags.pt"))
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32114, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
class tottodataset(Dataset):
  def __init__(self,df,tokenizer):
    self.sentence=df['sentence']
    self.table=df['table']
    self.tokenizer=tokenizer

  def __len__(self):
    return len(self.sentence)
  
  def __getitem__(self,idx):
    inp=(self.table[idx]+'</s>').replace("<page_title>", "<PAGESTART>").replace("</page_title>", "<PAGEEND>") \
                                    .replace("<section_title>", "<SECTIONSTART>").replace("</section_title>", "<SECTIONEND>") \
                                    .replace("<table>", "<TABLESTART>").replace("</table>", "<TABLEEND>") \
                                    .replace("<cell>", "<CELLSTART>").replace("</cell>", "<CELLEND>") \
                                    .replace("<col_header>", "<COLHEADERSTART>").replace("</col_header>", "<COLHEADEREND>") \
                                    .replace("<row_header>", "<ROWHEADERSTART>").replace("</row_header>", "<ROWHEADEREND>")
    out=self.sentence[idx]+'</s>'
    inp_tokens=self.tokenizer.encode_plus(inp, padding="max_length", max_length=MAXLENI, truncation=True)
    out_tokens=self.tokenizer.encode_plus(out, padding="max_length", max_length=MAXLENO, truncation=True)
    inp_id=inp_tokens.input_ids
    out_id=out_tokens.input_ids
    inp_mask=inp_tokens.attention_mask
    out_mask=out_tokens.attention_mask
    labels=out_tokens.input_ids.copy()
    labels=[-100  if x==self.tokenizer.pad_token_id else x for x in labels]

    return {
        "table_text":inp,
        "sentence":out,
        "input_ids":torch.tensor(inp_id, dtype=torch.long),
        "input_attention_mask":torch.tensor(inp_mask, dtype=torch.long),
        "decoder_input_ids":torch.tensor(out_id, dtype=torch.long),
        "decoder_attention_mask":torch.tensor(out_mask, dtype=torch.long),
        "labels":torch.tensor(labels, dtype=torch.long)
    }



In [ ]:
demo_dataset=tottodataset(df,tokenizer)
demo_dataloader=DataLoader(demo_dataset,
                            batch_size=1,
                            num_workers=2,
                            shuffle=False)

In [ ]:
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
inputs = next(iter(demo_dataloader))
generation_output = model.generate(inputs['input_ids'], return_dict_in_generate=True, output_scores=True)
expected=inputs["sentence"]
predicted=tokenizer.decode(generation_output[0][3],skip_special_tokens=True)

In [ ]:
import IPython
IPython.display.HTML(filename=f'/content/language_repo/example-3.html')

State (class),Vacator,Reason for change,Successor,Date of successor's formal installation
New Jersey (1),William Warren Barbour (R),"Died November 22, 1943. Successor was appointed until an election.",Arthur Walsh (D),"November 26, 1943"
Indiana (3),Frederick Van Nuys (D),"Died January 25, 1944. Successor was appointed until an election.",Samuel D. Jackson (D),"January 28, 1944"
Massachusetts (2),"Henry Cabot Lodge, Jr. (R)","Resigned February 3, 1944, to go on active duty in the US Army. Success was appointed until a special election.",Sinclair Weeks (R),"February 8, 1944"
Oregon (2),Charles L. McNary (R),"Died February 25, 1944. Successor was appointed and subsequently won special election",Guy Cordon (R),"March 4, 1944"
Washington (3),Homer Bone (D),"Resigned November 13, 1944, to become Judge of the U.S. Court of Appeals for the 9th Circuit.",Warren Magnuson (D),"December 14, 1944"
Indiana (3),Samuel D. Jackson (D),Appointee was not a candidate to finish the term. Successor was elected.,William E. Jenner (R),"November 7, 1944"
South Carolina (3),Ellison Durant Smith (D),"Died November 17, 1944.",Wilton E. Hall (D),"November 20, 1944"
New Jersey (1),Arthur Walsh (D),Appointee was not a candidate to finish the term. Successor was elected.,H. Alexander Smith (R),"December 7, 1944"
Massachusetts (2),Sinclair Weeks (R),Appointee was not a candidate to finish the term. Successor was elected.,Leverett Saltonstall (R),"January 4, 1945"


In [ ]:
print(f"Expected Sentence: {expected[3]}")
print(f"Predicted Sentence: {predicted}")

Expected Sentence: William Warren Barbour (R) served as Senate until November 22, 1943, then by Arthur Walsh (D) from November 26, 1943 and was succeeded by H. Alexander Smith (R), from December 7, 1944.</s>
Predicted Sentence: William Warren Barbour (R) served as Senate until November 22, 1943 and Arthur Walsh (D
